In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [ ]:
SBIN_1HR = yf.download('SBIN.NS',interval='1h',period='2y')

In [ ]:
SBIN_1HR

In [ ]:
df = SBIN_1HR.copy()
df_daily = df.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
}).dropna()


In [ ]:
df_daily

In [ ]:

# Calculate daily pivot points
df_daily['Pivot Point'] = (df_daily['High'] + df_daily['Low'] + df_daily['Close']) / 3
df_daily['TC'] = (df_daily['Pivot Point'] * 2) - df_daily['Low']  # Top Central Pivot
df_daily['BC'] = (df_daily['Pivot Point'] * 2) - df_daily['High']  # Bottom Central Pivot
df_daily['CPR Width'] = df_daily['TC'] - df_daily['BC']  # Calculate CPR width
df_daily['R1'] = df_daily['Pivot Point'] + df_daily['CPR Width'] / 2
df_daily['S1'] = df_daily['Pivot Point'] - df_daily['CPR Width'] / 2
df_daily['R2'] = df_daily['Pivot Point'] + df_daily['CPR Width']
df_daily['S2'] = df_daily['Pivot Point'] - df_daily['CPR Width']
df_daily['R3'] = df_daily['Pivot Point'] + df_daily['CPR Width'] * 1.5
df_daily['S3'] = df_daily['Pivot Point'] - df_daily['CPR Width'] * 1.5
df_daily['Date'] = df_daily.index.date
df_daily['%CPR'] = abs((df_daily['CPR Width'] / df_daily['Pivot Point']) * 100)
df['Date'] = df.index.date
df['Time'] = df.index.time
df['EMA_10'] = ta.ema(df['Close'], length=10)
df['EMA_20'] = ta.ema(df['Close'], length=20)


In [ ]:

# Calculate VWAP using pandas_ta
df['VWMA'] = ta.vwma(df['Close'], df['Volume'],length=10)

# Calculate MACD using pandas_ta
macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
df = pd.concat([df, macd], axis=1)

# Calculate RSI using pandas_ta
df['RSI'] = ta.rsi(df['Close'], length=14)



In [ ]:

# Filter the results to only include necessary columns
df_daily_cpr = df_daily.drop(columns=['Open', 'High', 'Low', 'Close'])

# Merge daily CPR data with the intraday data
df_merged = pd.merge(df, df_daily_cpr, on='Date', how='left')

# Debug: Print columns after merge
print("Columns after merge:", df_merged.columns)

# Drop the 'Date' column as it's no longer needed after the merge
df_merged['Datetime'] = pd.to_datetime(df_merged['Date'].astype(str) + ' ' + df_merged['Time'].astype(str))
df_merged.set_index('Datetime', inplace=True)


In [ ]:

# Ensure the 'TC' and 'BC' columns exist before forward filling
if 'TC' in df_merged.columns:
    df_merged['TC'] = df_merged['TC'].ffill()
else:
    print("Error: 'TC' column not found after merging. Check your data.")

if 'BC' in df_merged.columns:
    df_merged['BC'] = df_merged['BC'].ffill()
else:
    print("Error: 'BC' column not found after merging. Check your data.")

# Calculate CPR (Central Pivot Range)
df_merged['CPR'] = df_merged['TC'] - df_merged['BC']

# Ensure df_merged has a DatetimeIndex
if not isinstance(df_merged.index, pd.DatetimeIndex):
    df_merged.index = pd.to_datetime(df_merged.index)

# Filter the DataFrame for the desired trading time window
# df_merged = df_merged.between_time('09:20', '15:15')
df_merged

## Buy Entry Condition
1. RSI  > 60 and %CPR < 0.5%
2. today PP > Previous day PP
3. VWMA crosses above R1

# Exit Buy
1. Sl =0.5% or PP
2. After buying, if close and ema above R1, and close crosses below R1
3. Target s 1.5% or R3


In [ ]:

# Initialize the 'Entry_Price' and 'Exit_Price' columns
df_merged['Entry_Price'] = np.nan
df_merged['Exit_Price'] = np.nan

# Entry Conditions
df_merged['Buy_Entry'] = (
    (df_merged['Time'] >= pd.to_datetime('09:15:00').time()) &
    (df_merged['Time'] <= pd.to_datetime('15:00:00').time()) &
    (df_merged['RSI'] > 60) &
    (df_merged['%CPR'] < 0.5) &
    (df_merged['Pivot Point'] > df_merged['Pivot Point'].shift(1)) &
    (df_merged['VWMA'] > df_merged['R1']) &
    (df_merged['VWMA'].shift(1) <= df_merged['R1'].shift(1))  # VWMA crosses above R1
)


In [ ]:
df_merged[df_merged['Buy_Entry']]

In [ ]:
# Initialize the 'Entry_Price', 'Exit_Price', and other required columns
df_merged['Entry_Price'] = np.nan
df_merged['Exit_Price'] = np.nan
df_merged['Profit/Loss'] = np.nan
df_merged['Trade_Return'] = np.nan
df_merged['Cumulative_Return'] = np.nan

# Initialize additional KPI variables
trades = []
winning_trades = 0
losing_trades = 0
max_drawdown = 0
cumulative_return = 1



In [ ]:
# Loop through the data to manage trades
for i in range(1, len(df_merged)):
    if df_merged.iloc[i]['Buy_Entry']:
        entry_price = df_merged.iloc[i]['Close']
        df_merged.iloc[i, df_merged.columns.get_loc('Entry_Price')] = entry_price
        
        # Monitor for Exit
        for j in range(i+1, len(df_merged)):
            df_merged.iloc[j, df_merged.columns.get_loc('Entry_Price')] = entry_price  # Carry forward the entry price for exit conditions
            
            # Exit Conditions
            exit_condition = (
                (df_merged.iloc[j]['Close'] < df_merged.iloc[j]['R1']) & 
                (df_merged.iloc[j]['Close'] > entry_price * 0.985) &  # SL at 0.5% below entry
                (df_merged.iloc[j]['Close'] < entry_price * 1.015)    # Target at 1.5% above entry
            )
            
            time_condition = df_merged.iloc[j]['Time'] >= pd.to_datetime('15:00:00').time()
            
            if exit_condition.any() or time_condition:
                exit_price = df_merged.iloc[j]['Close']
                df_merged.iloc[j, df_merged.columns.get_loc('Exit_Price')] = exit_price
                
                # Calculate Profit/Loss
                profit_loss = exit_price - entry_price
                df_merged.iloc[j, df_merged.columns.get_loc('Profit/Loss')] = profit_loss
                
                # Calculate Trade Return
                trade_return = (exit_price / entry_price) - 1
                df_merged.iloc[j, df_merged.columns.get_loc('Trade_Return')] = trade_return
                
                # Update cumulative return
                cumulative_return *= (1 + trade_return)
                df_merged.iloc[j, df_merged.columns.get_loc('Cumulative_Return')] = cumulative_return
                
                # Track trades for KPIs
                trades.append(profit_loss)
                if profit_loss > 0:
                    winning_trades += 1
                else:
                    losing_trades += 1

                # Calculate drawdown
                peak = max(cumulative_return, 1)
                drawdown = (peak - cumulative_return) / peak
                max_drawdown = max(max_drawdown, drawdown)
                
                break


In [ ]:
# Calculate KPIs
total_trades = len(trades)
total_profit = sum(trades)
average_trade_return = np.mean(df_merged['Trade_Return'].dropna())
win_loss_ratio = winning_trades / max(losing_trades, 1)
cumulative_return_pct = (cumulative_return - 1) * 100

# Print KPIs
print("Total Trades:", total_trades)
print("Total Profit/Loss:", total_profit)
print("Average Trade Return:", average_trade_return)
print("Win/Loss Ratio:", win_loss_ratio)
print("Cumulative Return (%):", cumulative_return_pct)
print("Max Drawdown:", max_drawdown)

# Display the first few rows of the resulting DataFrame with KPIs
df_merged[df_merged['Buy_Entry']]




In [ ]:
df_merged[df_merged['Exit_Price'].notna()]


In [ ]:

# Calculate EMA using pandas_ta

# Generate buy/sell signals based on conditions (example condition below)
df_merged['Signal'] = 0


# Create a DataFrame for trades
trades = []
position = 0
entry_price = 0

for index, row in df_merged.iterrows():
    if row['Signal'] == 1 and position == 0:
        position = 1
        entry_price = row['Close']
        trades.append({'Time': index, 'Entry Price': entry_price})
    elif row['Signal'] == -1 and position == 1:
        position = 0
        exit_price = row['Close']
        profit = exit_price - entry_price
        trades[-1].update({'Exit Price': exit_price, 'Profit': profit})

trades_df = pd.DataFrame(trades)



In [ ]:
trades_df